In [1]:
import json
import bz2
import sys
import os
import pandas as pd
import time

In [3]:
WIKI_DATA_FILTERED = 'filtered_politician.json.bz2'
QUOTEBANK_POLITICIANS_WORLD= 'quotebank_politicians_world.json.bz2'
QUOTEBANK_POLITICIANS_USA= 'quotebank_politicians_USA.json.bz2'
quotebank_dir ='../data/'
print(type(os.listdir(quotebank_dir)))
quotebank_files=list( os.path.join(quotebank_dir,x) for x in os.listdir(quotebank_dir))
print(quotebank_files)

<class 'list'>
['../data/quotes-2015.json-003.bz2', '../data/quotes-2016.json-002.bz2', '../data/quotes-2017.json.bz2', '../data/quotes-2018.json-006.bz2', '../data/quotes-2019.json-004.bz2', '../data/quotes-2020.json.bz2']


In [4]:
qids_df=pd.read_json(WIKI_DATA_FILTERED, lines=True)
qids_df.head()

,qid,name,nationality
0,Q207,George W. Bush,Q30
1,Q946,Donald Tusk,Q36
2,Q1058,Narendra Modi,Q668
3,Q1253,Ban Ki-moon,Q884
4,Q3996,V. P. Kalairajan,Q668


In [5]:
qids_us_df = qids_df[qids_df['nationality']=='Q30']
qids_us_df.head()

,qid,name,nationality
0,Q207,George W. Bush,Q30
5,Q4496,Mitt Romney,Q30
7,Q11124,Stephen Breyer,Q30
9,Q11674,David Paterson,Q30
23,Q22201,Jim DeMint,Q30


In [6]:
qid_list = list(qids_df.qid)
qid_list_us = list(qids_us_df.qid)

In [7]:
CHUNKSIZE = 100000
try:
    os.remove(QUOTEBANK_POLITICIANS_WORLD)
    os.remove(QUOTEBANK_POLITICIANS_USA)
except:
    pass
total_time, sum_quotes_world, sum_quotes_usa, chunk_number=0,0,0,0

with bz2.open(QUOTEBANK_POLITICIANS_WORLD, 'wb') as bzout_world:
    with bz2.open(QUOTEBANK_POLITICIANS_USA, 'wb') as bzout_usa:
        for file in quotebank_files:
            with pd.read_json(file, lines=True, compression='bz2', chunksize=CHUNKSIZE) as df_reader:
                for chunk in df_reader:
                    t1=time.time()
                    chunk_not_empty = chunk['qids'].apply(lambda x: len(x)) != 0
                    chunk_politician = chunk[chunk_not_empty]['qids'].apply(lambda x: x[0]).isin(qid_list)
                    chunk_politician_us = chunk[chunk_not_empty]['qids'].apply(lambda x: x[0]).isin(qid_list_us)
                    quote_world = chunk[chunk_not_empty & chunk_politician]
                    quote_usa = chunk[chunk_not_empty & chunk_politician_us]
                    quote_world.to_json(path_or_buf=bzout_world,orient='records',lines=True)
                    quote_usa.to_json(path_or_buf=bzout_usa,orient='records',lines=True)
                    sum_quotes_world += len(quote_world)
                    sum_quotes_usa += len(quote_usa)
                    t2=time.time()
                    dt=t2-t1
                    total_time += dt
                    chunk_number += 1
                    print(chunk_number)
                    print("Dumped {} political quotations and {} US political quotations among {} quotations [quotations/s: {:.2f}]".format(len(quote_world), len(quote_usa), chunk_number*CHUNKSIZE, sum_quotes_world / dt))
                    

1
Dumped 12139 political quotations and 5908 US political quotations among 100000 quotations [quotations/s: 6615.09]
2
Dumped 12129 political quotations and 5825 US political quotations among 200000 quotations [quotations/s: 11899.81]
3
Dumped 11968 political quotations and 5743 US political quotations among 300000 quotations [quotations/s: 21156.30]
4
Dumped 12074 political quotations and 5744 US political quotations among 400000 quotations [quotations/s: 27530.19]
5
Dumped 12169 political quotations and 5884 US political quotations among 500000 quotations [quotations/s: 30213.61]
6
Dumped 12064 political quotations and 5819 US political quotations among 600000 quotations [quotations/s: 39714.85]
7
Dumped 12210 political quotations and 5909 US political quotations among 700000 quotations [quotations/s: 48012.25]
8
Dumped 11976 political quotations and 5707 US political quotations among 800000 quotations [quotations/s: 52578.46]
9
Dumped 12177 political quotations and 5807 US political

In [238]:
print(f'WORLD: total quotes: {sum_quotes_world} in {total_time}s')
print(f'USA: total quotes: {sum_quotes_usa} in {total_time}s')

WORLD: total quotes: 812900 in 88.9895088672638s
USA: total quotes: 401991 in 88.9895088672638s
